# Opioid Data - Standardize Timeseries
HW #2 Part 2 - Timeseries.  
Use all rows per patient from about 30 consecutive days.
Standardize all of it.

## Preprocessing
Input characteristics: 
Patient files are in one of two directories: R or NR.
Each patient is represented by one CSV file.
Each row of each CSV contains readings from one day.
Days are in order and mostly sequential (at least one day is missing in one patient).
Most patients have 30 days but some have less.

Loaded data structure characteristics:
Labels columns for cohort (R or NR), patient (2005_S3), date.
Labels rows are num_cohorts * num_patients * num_dates.
Features columns are measurements without dates.
Features rows are same as Labels rows.

In [2]:
from os import listdir
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import time

In [4]:
pathR='data/ChunkedData_R/'
pathN='data/chunkedData_NR/'
filesR = listdir(pathR)
filesN = listdir(pathN)
print(len(filesR)," files of type R")
print(len(filesN)," files of type N")
CLASS_SEPARATOR=13  # data[:13] vs data[13:]
WITH_VARIANCE_COLUMNS=True   # Use mean and variance per patient

14  files of type R
26  files of type N


In [38]:
labels_list=[]
features_list=[]
# Read one CSV file. 
# Load global lists
def load_patient (filepath,cohort,patient_name):
    patient_recs = pd.read_csv(filepath)
    rows,cols = patient_recs.shape
    print(rows,cols)
    for r in range(0,rows):
        record = patient_recs.iloc[r]
        dt=record['Date']
        label = (cohort,patient_name,dt)
        labels_list.append(label)


In [39]:
load_patient('data/chunkedData_NR/Daily_2002_S1.csv','N','2002_S1')
labels_list

29 260


[('N', '2002_S1', '2020-01-03'),
 ('N', '2002_S1', '2020-01-04'),
 ('N', '2002_S1', '2020-01-05'),
 ('N', '2002_S1', '2020-01-06'),
 ('N', '2002_S1', '2020-01-07'),
 ('N', '2002_S1', '2020-01-08'),
 ('N', '2002_S1', '2020-01-09'),
 ('N', '2002_S1', '2020-01-10'),
 ('N', '2002_S1', '2020-01-11'),
 ('N', '2002_S1', '2020-01-12'),
 ('N', '2002_S1', '2020-01-13'),
 ('N', '2002_S1', '2020-01-14'),
 ('N', '2002_S1', '2020-01-15'),
 ('N', '2002_S1', '2020-01-16'),
 ('N', '2002_S1', '2020-01-17'),
 ('N', '2002_S1', '2020-01-18'),
 ('N', '2002_S1', '2020-01-19'),
 ('N', '2002_S1', '2020-01-20'),
 ('N', '2002_S1', '2020-01-21'),
 ('N', '2002_S1', '2020-01-22'),
 ('N', '2002_S1', '2020-01-23'),
 ('N', '2002_S1', '2020-01-24'),
 ('N', '2002_S1', '2020-01-25'),
 ('N', '2002_S1', '2020-01-26'),
 ('N', '2002_S1', '2020-01-27'),
 ('N', '2002_S1', '2020-01-28'),
 ('N', '2002_S1', '2020-01-29'),
 ('N', '2002_S1', '2020-01-30'),
 ('N', '2002_S1', '2020-01-31')]

In [3]:
# Read directory of CSV files (R or NR). 
# Given directory, load all the patients in that directory.
# We use filenames as patient names.
def load_cohort (directory):
    file_names = listdir(directory)
    patient_names = []
    patient_records = []
    for fp in file_names:
        dfp = directory+fp
        one_patient = load_patient(dfp)
        one_name = fp.split('.')[0]  # strip away .csv suffix
        one_name = one_name[6:]    # strip away Daily_ prefix
        patient_names.append(one_name)
        patient_records.append(one_patient)
    return patient_names,patient_records

In [4]:
namesR,dataR = load_cohort(pathR)
namesN,dataN = load_cohort(pathN)

In [5]:
dataAll = dataR + dataN
rowsR = len(dataR)
rowsN = len(dataN)
rowsAll = len(dataAll)
print("Number of patients (R,N,total):",rowsR,rowsN,rowsAll)

# Visit all patients in order.
# Either print the dates or remove the dates.
#for p in range(rowsAll):
#    patient_recs = dataAll[p]
#    #print(patient_recs['Date'])
#    patient_recs.drop(['Date'],axis=1)

Number of patients (R,N,total): 14 26 40


We will assume timeseries records are equally spaced at one per day.
This is mostly true with a few abberations.
For example, R patient 2060_S3 is missing the record between 2020-06-01 and 2020-06-03.

We will align all patient records by measuring days-since-start.
We will ignore the specific dates which can be in different months for different patients.
Thus, we are assuming month of year has no effect on the response variable R/NR.

## Scaling and covariance
Normalize by subtracting the column mean from every column value.  
Since columns have widely different numerical ranges,   
also normalize by making each column have unit variance.  
Note: without normalization, the covariance plot would be all black except for the few features with large absolute values.

In [6]:
# Standardize features by shifting the mean to zero and scaling to unit variance.
# Subtract the mean and divide by the std.dev: z = (x - u) / s
def scale_features(X):
    s = StandardScaler()
    z = s.fit_transform(X)
    return z
scaledMeans = scale_features(meansAll)
column_names = meansAll.columns

NameError: name 'meansAll' is not defined

In [ ]:
print("Column Names:")
print(column_names)
print("Scaled Means & Variances:")
scaledMeans

In [ ]:
pathR = None
pathN = None
filesR = None
filesN = None
meansR = None
meansN = None
meansAll = None